In [3]:
from project import *
import csv
import mysql.connector as sql
from mysql.connector import Error

In [4]:
# get connection 
dbConn = create_connection("localhost", "root", "123456", "progDB")
cursor = dbConn.cursor()
create_database(cursor, "progDB")

Database created successfully


In [24]:
# create the tables from file
# NOTE: need to rework for the if not exists  
def create_tables_from_file_m(cursor, filename, connection):
    try:
        with open(filename,'r') as file:
            sql_script = file.read()
            
        for result in cursor.execute(sql_script, multi=True):
            pass
    except Error as e:
        print(f"The error '{e} occurred'")
    
    connection.commit()
    

## General Database Management

In [12]:
tables = ["Programs", "Departments", "Faculty", "Courses",
          "Semesters", "CourseSections", "LearningObjectives", 
          "ProgramObjectives", "SectionObjectives"]

def clear_database(cursor, connection, tableList):
    try:
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
        for table in tableList:
            sqlcom = "DROP TABLE IF EXISTS %s" % (table)
            cursor.execute(sqlcom)
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    except Error as e:
        print(f"The error '{e} occurred'")
    connection.commit()

In [19]:
def enter_program_info(cursor, connection, prog_name, pc_id, pc_name, pc_email, dept_code):
    '''Check to see if program coordinator exists, if not add them to the faculty table'''
    try:
        cursor.execute("SELECT * FROM Faculty WHERE FacultyID = %s", pc_id)
        result = cursor.fetchall()
    except Error as e:
        print(f"Error executing Faculty select: {e}")
    
    if not cursor.fetchall():
        #make new faculty
        enter_faculty_info(cursor, connection, pc_id, pc_name, pc_email, dept_code, fac_rank=0)
    try:
        # Insert data into the Programs table
        cursor.execute("""
            INSERT INTO Programs (ProgramName, ProgramCoordinatorID, ProgramCoordinatorName, ProgramCoordinatorEmail, DepartmentCode)
            VALUES (?, ?, ?, ?, ?)
        """, (prog_name, pc_id, pc_name, pc_email, dept_code))

        # Commit the transaction
        connection.commit()

        print(f"Data for program '{prog_name}' successfully inserted into the Programs table.")
    except Error as e:
        print(f"Error inserting program data: {e}")
    

def enter_department_info(cursor, connection, dep_code, dep_name):
    try:
        # Insert data into the Departments table
        cursor.execute("""
            INSERT INTO Departments (DepartmentCode, DepartmentName)
            VALUES (?, ?)
        """, (dep_code, dep_name))

        # Commit the transaction
        connection.commit()

        print(f"Data for department '{dep_name}' successfully inserted into the Departments table.")
    except Error as e:
        print(f"Error inserting department data: {e}")

def enter_faculty_info(cursor, connection, fac_id, fac_name, fac_email, dep_code, fac_rank=0):
    try:
        # Insert data into the Faculty table
        cursor.execute("""
            INSERT INTO Faculty (FacultyID, FacultyName, FacultyEmail, FacultyRank, DepartmentCode)
            VALUES (?, ?, ?, ?, ?)
        """, (fac_id, fac_name, fac_email, fac_rank, dep_code))

        # Commit the transaction
        connection.commit()

        print(f"Data for faculty '{fac_name}' successfully inserted into the Faculty table.")
    except Error as e:
        print(f"Error inserting faculty data: {e}")

def enter_course_info(cursor, connection, co_id, co_title, co_desc, dep_code):
    try:
        # Insert data into the Courses table
        cursor.execute("""
            INSERT INTO Courses (CourseID, CourseTitle, CourseDescription, DepartmentCode)
            VALUES (?, ?, ?, ?)
        """, (co_id, co_title, co_desc, dep_code))

        # Commit the transaction
        connection.commit()

        print(f"Data for course '{co_title}' successfully inserted into the Courses table.")
    except Error as e:
        print(f"Error inserting course data: {e}")
    